In [1]:
import cv2
import numpy as np

In [2]:
orange = np.uint8([[[0, 165, 255]]])
ycrcb_orange = cv2.cvtColor(orange, cv2.COLOR_BGR2YCR_CB)
print(ycrcb_orange)
ycrcb_green = cv2.cvtColor(np.uint8([[[0, 255, 0]]]), cv2.COLOR_BGR2YCrCb)
print(ycrcb_green)

[[[173 186  30]]]
[[[150  21  43]]]


In [3]:
black = cv2.cvtColor(np.uint8([[[255,255,255]]]), cv2.COLOR_BGR2YCrCb)
print(black)

[[[255 128 128]]]


In [4]:
center_orange = np.array([173, 186, 30])
lower_orange = center_orange - np.array([173, 186-141, 30])
upper_orange = center_orange + np.array([255-173, 230-186, 80])
# threshold = 256

In [8]:
cap = cv2.VideoCapture(1)

while 1:
    ret, img = cap.read()
    
    cv2.imshow("yes", img)
    
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    
#     h, s, v = cv2.split(hsv)
#     cv2.equalizeHist(h, h)
#     cv2.merge((h, s, v), hsv)
    
    cv2.imshow("ycrcb", ycrcb)
    
#     mask = cv2.GaussianBlur(mask, (15, 1), cv2.BORDER_DEFAULT)
    
#     cv2.imshow("blur", mask)
    
    mask = cv2.inRange(ycrcb, lower_orange, upper_orange)
    
#     def maskify(luke):
#         new = []
#         for x in range(len(luke)):
#             new.append([])
#             for y in range(len(luke[0])):
#                 val = 0
#                 diff = luke[x][y] - center_orange
#                 print(luke[x][y][1])
#                 if (diff[1] ** 2 + diff[2] ** 2 <= threshold ** 2):
#                     val = 255
#                 new[x].append(val)
#         return np.array(new, dtype=np.uint8)
                    
#     mask = maskify(ycrcb)
    
    cv2.imshow("mask", mask)
    
    mask = cv2.GaussianBlur(mask, (5, 15), cv2.BORDER_DEFAULT)
    
    cv2.imshow("blur", mask)
    
    res = cv2.bitwise_and(img, img, mask = mask)
    
    contours, he = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    res = cv2.drawContours(res, contours, -1, (0 , 255, 0), 3)
    
    cv2.imshow("res", res)
    
    def weightC(c):
        x,y,w,h = cv2.boundingRect(c)
#         box = cv2.boxPoints(rect)
#         box = np.int0(box)
        return w
    
    cPrime = sorted([(weightC(c), i) for i, c in zip(range(len(contours)),contours)], reverse = True)
    
#     count = 0
#     for cP in cPrime:
#         if cP[0] >= 200:
#             count += 1
            
#     boxes = []
#     for i in range(count):
#         rect = cv2.boundingRect(contours[cPrime[i][1]])
#         box = cv2.boxPoints(rect)
#         box = np.int0(box)
#         boxc = box[:]
#         def avg(b, i):
#             return sum(b[i]) / len(b)
        
#         avgY = avg(boxc, 1)
# #         boxes.append(avgY)
#         boxes.append(boxc[1])
#         cv2.drawContours(res,[box],0,(255,0,0),2)
    
#     print(len(boxes))
    
    if (cPrime != []):
        x,y,w,h = cv2.boundingRect(contours[cPrime[0][1]])
#         box = cv2.boxPoints(rect)
#         box = np.int0(box)
        cv2.rectangle(res, (x,y), (x+w,y+h), (255,0,0), 2)
        cv2.putText(res, f"aspect ratio: {h/w}", (50,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
        if (w >= 100):
            if (.5 < h / w):
                cv2.putText(res, "# of rings: 4", (50,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
            else:
                cv2.putText(res, "# of rings: 1", (50,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        else:
            cv2.putText(res, "# of rings: 0", (50,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    else:
        cv2.putText(res, "# of rings: 0", (50,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        
    
    cv2.imshow("res2", res)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
